In [1]:
!pwd

/content


In [2]:
!python --version

Python 3.10.12


In [3]:
#Run once per session (files get deleted after runtime resource terminates):

!git clone https://github.com/openai/shap-e
%cd shap-e
!pip install -e .


Cloning into 'shap-e'...
remote: Enumerating objects: 336, done.
remote: Counting objects: 100% (260/260), done.
remote: Compressing objects: 100% (241/241), done.
remote: Total 336 (delta 41), reused 217 (delta 18), pack-reused 76
Receiving objects: 100% (336/336), 11.72 MiB | 16.84 MiB/s, done.
Resolving deltas: 100% (44/44), done.
/content/shap-e
Obtaining file:///content/shap-e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-ktedl6l6/clip_ed84358d76294c69987155e9f4812543
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-ktedl6l6/clip_ed84358d76294c69987155e9f4812543
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.

In [4]:
!pip install trimesh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 689.0/689.0 kB 7.9 MB/s eta 0:00:00


In [5]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

import os

from PIL import Image

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|███████████████████████████████████████| 890M/890M [00:11<00:00, 83.6MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

In [8]:
batch_size = 4 #Default 4
guidance_scale = 7.0 #Just like CFG Scale in Stable Diffusion
#prompt = input("Enter what you want Shap-e to generate here:")
prompt = "fantasy low poly goblin."

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [9]:
import imageio
render_mode = 'stf' # you can change this to 'stf' or 'nerf'
size = 128 # this is the size of the renders; higher values take longer to render. 64

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))


    image_list = []
    for i in range(len(images)):
      image = images[i]
      image_list.append(image)
    imageio.mimsave(f"color_gradient_{i}.gif", image_list)



/content/shap-e/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/content/shap-e/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(


HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAALp2NrR1NLJvM61rMqZrMJ5rL6JlLphkLJZgK5BaKYZeJoNXJI…

HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAAHOIJGaCIWh/IlmCHld8HGZ5IF52HU95GVB0Gkh+HER7GkB6GU…

HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAAIBwPXhrOnJmNmdiLlZfGVlbIVRcGFtWJlVUGFBcFU5bFE1ZFE…

HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAAI6MSHqDPHV8PWZ4L2duNmBvL1tvKFltJlRtLF9nL1dpJ1RqKF…

In [86]:
# Example of saving the latents as meshes.

from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    t = decode_latent_mesh(xm, latent).tri_mesh()
    with open(f'example_mesh_{i}.ply', 'wb') as f:
      t.write_ply(f)
    with open(f'example_mesh_{i}.obj', 'w') as f:
      t.write_obj(f)



/content/shap-e/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/content/shap-e/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(


In [11]:
im = Image.open("color_gradient_19.gif")
# Get the color of the first pixel in the first frame
color = im.getpixel((0, 0))


In [64]:
!pip install open3d


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 85.7 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [84]:
import open3d as o3d
import numpy as np
from PIL import Image

# Load the color gradient image
gradient = Image.open("color_gradient.gif")

# Convert the color gradient image to a numpy array
color = np.array(gradient).astype(np.float32) / 255.0

# Create a random depth map with the same shape as the color image
depth = np.random.uniform(0.0, 1.0, size=color.shape[:2]).astype(np.float32)

# Create a custom camera intrinsic with the same resolution as the color image
intrinsics = o3d.camera.PinholeCameraIntrinsic(color.shape[1], color.shape[0], 525.0, 525.0, color.shape[1] // 2, color.shape[0] // 2)

# Convert the color and depth arrays to Open3D Image objects
color_image = o3d.geometry.Image(np.asarray(color))
depth_image = o3d.geometry.Image(np.asarray(depth))

# Get the data buffers for the color and depth images
color_data_ptr = color_image.numpy().ctypes.data
depth_data_ptr = depth_image.numpy().ctypes.data

# Create Open3D CUDA images from the data buffers
color_cuda = o3d.cuda.pybind.geometry.Image.create_from_color_buffer(color_data_ptr, color_image.width, color_image.height)
depth_cuda = o3d.cuda.pybind.geometry.Image.create_from_depth_buffer(depth_data_ptr, depth_image.width, depth_image.height)

# Create an Open3D RGBD image from the color and depth images
rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(color_cuda, depth_cuda, depth_scale=1000.0, depth_trunc=3.0, convert_rgb_to_intensity=False, intrinsic=intrinsics)

# Create a point cloud from the RGBD image
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsics)

# Estimate normals for the point cloud
pcd.estimate_normals()

# Create a mesh from the point cloud
mesh, _ = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd)

# Assign color to the mesh
mesh.paint_uniform_color(color.reshape(-1, 3))

# Save the mesh to a PLY file
o3d.io.write_point_cloud("mesh.ply", mesh)


AttributeError: 'open3d.cuda.pybind.geometry.Image' object has no attribute 'numpy'

In [85]:
@torch.no_grad()
def decode_render(
    xm: Union[Transmitter, VectorDecoder],
    latent: torch.Tensor,
    cameras: DifferentiableCameraBatch,
    rendering_mode: str = "stf",
):
    decoded = xm.renderer.render_views(
        AttrDict(cameras=cameras),
        params=(xm.encoder if isinstance(xm, Transmitter) else xm).bottleneck_to_params(
            latent[None]
        ),
        options=AttrDict(rendering_mode=rendering_mode, render_with_direction=False),
    )
    arr = decoded.channels.clamp(0, 255).to(torch.uint8)[0].cpu().numpy()
    return decoded, [Image.fromarray(x) for x in arr]


render_mode = 'stf' # you can change this to 'stf' for mesh rendering
size = 2  # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    decoded, images = decode_render(xm, latent, cameras, rendering_mode=render_mode)
    assert len(decoded['meshes']) == 1
    mesh = decoded['meshes'][0]
    trimesh.exchange.export.export_mesh(trimesh.Trimesh(
        mesh.vertices.cpu().numpy(),
        mesh.faces.cpu().numpy(),
        vertex_colors=mesh.vertex_colors.cpu().numpy()
    ), "barrel-gen.glb")

NameError: name 'Union' is not defined